# Solve for Gaussian approximations using optimization

In [1]:
%load_ext autoreload
%autoreload 2
# this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

from cleanplots import *
from tqdm import tqdm
from information_estimation import *
from image_utils import *
from gaussian_process_utils import *

from led_array.bsccm_utils import *
from bsccm import BSCCM
from jax import jit
import numpy as onp
import jax.numpy as np

bsccm = BSCCM('/home/hpinkard_waller/data/BSCCM/')

Opening BSCCM
Opened BSCCM


In [2]:
# load images, extract patches, and compute cov mats
edge_crop = 32
patch_size = 20
num_images = 20000
num_patches = 1000
channel = 'LED119'
eigenvalue_floor = 1e0

images = load_bsccm_images(bsccm, channel=channel, num_images=num_images, edge_crop=edge_crop, median_filter=False)

## Search through hyperparameter combos 

In [3]:
learning_rates = np.logspace(1, -8, 20)
batch_sizes = np.linspace(2, 50, 20).astype(int)
momentums = np.linspace(0, 0.999, 20)

# generate tuples of random hyperparameters
hyperparameter_tuples = []
for i in range(10000):
    lr = onp.random.choice(learning_rates)
    bs = onp.random.choice(batch_sizes)
    m = onp.random.choice(momentums)
    hyperparameter_tuples.append((lr, bs, m))

results = {}
for i, (learning_rate, batch_size, momentum) in enumerate(hyperparameter_tuples):
    best_hp_loss = np.inf

    patches = extract_patches(images, patch_size, num_patches=num_patches, seed=i)
    best_cov_mat, cov_mat_initial, mean_vec, best_loss = run_optimization(patches, momentum, learning_rate, batch_size, eigenvalue_floor=1e-3)

    if best_loss < best_hp_loss:
        best_hp_loss = best_loss
        best_hp = (learning_rate, batch_size, momentum)
        
    # collect results
    results[(learning_rate, batch_size, momentum)] = best_loss

    # print hyperparameters and their best loss
    print(f"best loss: {best_loss:.2f}\t\tLearning rate: {learning_rate:.3e}, Batch size: {batch_size}, Momentum: {momentum:.3e}")

Initial loss:  1405663.069315428
best loss: 1213584.55		Learning rate: 1.624e-03, Batch size: 47, Momentum: 4.732e-01
Initial loss:  1236283.0570866575
best loss: 81392.61		Learning rate: 2.637e-07, Batch size: 39, Momentum: 1.577e-01
Initial loss:  121888.29746743452
best loss: 1783.17		Learning rate: 2.336e-06, Batch size: 17, Momentum: 7.887e-01
Initial loss:  601628.0376138617
best loss: 3075.61		Learning rate: 2.069e-05, Batch size: 2, Momentum: 4.732e-01
Initial loss:  257936.13077799967
best loss: 3626.99		Learning rate: 1.833e-04, Batch size: 37, Momentum: 9.464e-01
Initial loss:  96484.46934036847
best loss: 3870.65		Learning rate: 1.624e-03, Batch size: 4, Momentum: 7.887e-01
Initial loss:  299957.18864992226
best loss: 5684.87		Learning rate: 3.360e+00, Batch size: 44, Momentum: 9.464e-01
Initial loss:  646669.6595893062
best loss: 225296.33		Learning rate: 2.069e-05, Batch size: 17, Momentum: 0.000e+00
Initial loss:  1198738.0557458473
best loss: 5126.40		Learning rate: 3.7

Traceback (most recent call last):
  File "/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_12112/3563418900.py", line 18, in <module>
    best_cov_mat, cov_mat_initial, mean_vec, best_loss = run_optimization(patches, momentum, learning_rate, batch_size, eigenvalue_floor=1e-3)
  File "/home/hpinkard_waller/GitRepos/EncodingInformation/gaussian_process_utils.py", line 488, in run_optimization
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1

In [4]:
# print the hyperparameters ranked from best to worst
sorted_results = sorted(results.items(), key=lambda x: x[1])
for hp, loss in sorted_results:
    print(f"best loss: {loss:.2f}\t\tLearning rate: {hp[0]:.3e}, Batch size: {hp[1]}, Momentum: {hp[2]:.3e}")

best loss: 1677.30		Learning rate: 1.000e-08, Batch size: 2, Momentum: 3.681e-01
best loss: 1710.50		Learning rate: 1.000e+01, Batch size: 2, Momentum: 7.887e-01
best loss: 1712.38		Learning rate: 1.274e-01, Batch size: 2, Momentum: 2.103e-01
best loss: 1713.47		Learning rate: 1.438e-02, Batch size: 2, Momentum: 7.887e-01
best loss: 1716.37		Learning rate: 6.158e-05, Batch size: 2, Momentum: 6.309e-01
best loss: 1716.63		Learning rate: 1.624e-03, Batch size: 2, Momentum: 1.052e-01
best loss: 1725.65		Learning rate: 5.456e-04, Batch size: 2, Momentum: 1.577e-01
best loss: 1733.78		Learning rate: 2.637e-07, Batch size: 4, Momentum: 3.681e-01
best loss: 1741.91		Learning rate: 1.000e-08, Batch size: 4, Momentum: 9.990e-01
best loss: 1744.23		Learning rate: 1.129e+00, Batch size: 12, Momentum: 5.784e-01
best loss: 1747.73		Learning rate: 2.637e-07, Batch size: 7, Momentum: 8.413e-01
best loss: 1752.28		Learning rate: 8.859e-08, Batch size: 7, Momentum: 7.887e-01
best loss: 1752.41		Learnin